<a href="https://colab.research.google.com/github/dhirajkrgupta/classical-models/blob/main/crossvalidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textstat
!pip install gdown  # only if not already installed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score,roc_curve,auc
from sklearn.preprocessing import StandardScaler
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from textstat import flesch_reading_ease, flesch_kincaid_grade
import json
import matplotlib.pyplot as plt

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

# **Step 1: Data Preparation**

In [ ]:
def load_and_prepare_data(json_file_path):
    """
    Load the dataset and create labels for human vs AI classification
    """
    data = []

    # Read the JSON dataset
    with open(json_file_path, 'r') as f:
        for line_num, line in enumerate(f, 1):
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Skipping line {line_num} due to JSONDecodeError: {e}")
                continue  # Skip to the next line if decoding fails
    #make a table from the data
    df = pd.DataFrame(data)

    # Create the classification dataset
    classification_data = []
    #iterate through each row of the table df
    for _, row in df.iterrows():
        # Human review (label = 1)
        human_review = row.get('full_human_review', '')
        if human_review and str(human_review).strip():  # Check if not empty
            classification_data.append({
                'text': str(human_review).strip(),
                'label': 1,  # Human = 1
                'source': 'human'
            })

        # AI/Machine review (label = 0)
        machine_review = row.get('machine_review', '')
        if machine_review and str(machine_review).strip():  # Check if not empty
            classification_data.append({
                'text': str(machine_review).strip(),
                'label': 0,  # AI = 0
                'source': 'ai'
            })

    return pd.DataFrame(classification_data)

# **Step 2: Feature Engineering**

In [ ]:
class ReviewFeatureExtractor:
    """
    Extract various features from review texts
    """

    def __init__(self):
        # Initialize sentiment analyzer
        nltk.download('vader_lexicon', quiet=True)
        self.sia = SentimentIntensityAnalyzer()

    def extract_linguistic_features(self, text):
        """Extract linguistic and stylistic features"""
        features = {}

        # Handle empty, None, or invalid text
        if not text or not isinstance(text, str) or pd.isna(text):
            text = ""

        text = str(text).strip()

        # Basic text statistics
        features['char_count'] = len(text)
        words = text.split() if text else []
        features['word_count'] = len(words)
        sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()] if text else []
        features['sentence_count'] = max(len(sentences), 1)
        features['avg_word_length'] = np.mean([len(word) for word in words]) if words else 0
        features['avg_sentence_length'] = features['word_count'] / features['sentence_count']

        # Replace NaN values with 0
        if np.isnan(features['avg_word_length']):
            features['avg_word_length'] = 0

        # Readability scores (with error handling)
        try:
            if text and len(text) > 10:
                features['flesch_reading_ease'] = flesch_reading_ease(text)
                features['flesch_kincaid_grade'] = flesch_kincaid_grade(text)
            else:
                features['flesch_reading_ease'] = 0
                features['flesch_kincaid_grade'] = 0
        except:
            features['flesch_reading_ease'] = 0
            features['flesch_kincaid_grade'] = 0

        # Punctuation and formatting
        text_len = max(len(text), 1)
        features['exclamation_ratio'] = text.count('!') / text_len
        features['question_ratio'] = text.count('?') / text_len
        features['comma_ratio'] = text.count(',') / text_len
        features['bullet_points'] = text.count('*') + text.count('-')

        # Sentiment analysis
        try:
            if text:
                sentiment_scores = self.sia.polarity_scores(text)
                features['sentiment_pos'] = sentiment_scores['pos']
                features['sentiment_neg'] = sentiment_scores['neg']
                features['sentiment_neu'] = sentiment_scores['neu']
                features['sentiment_compound'] = sentiment_scores['compound']
            else:
                features['sentiment_pos'] = 0
                features['sentiment_neg'] = 0
                features['sentiment_neu'] = 0.5
                features['sentiment_compound'] = 0
        except:
            features['sentiment_pos'] = 0
            features['sentiment_neg'] = 0
            features['sentiment_neu'] = 0.5
            features['sentiment_compound'] = 0

        # Academic writing indicators
        if text:
            features['first_person_pronouns'] = len(re.findall(r'\b(I|me|my|we|us|our)\b', text, re.IGNORECASE))
            features['passive_voice_indicators'] = len(re.findall(r'\b(is|are|was|were|been|being)\s+\w+ed\b', text))
            features['academic_phrases'] = len(re.findall(r'\b(however|furthermore|moreover|therefore|consequently|nevertheless)\b', text, re.IGNORECASE))
        else:
            features['first_person_pronouns'] = 0
            features['passive_voice_indicators'] = 0
            features['academic_phrases'] = 0

        return features

    def extract_all_features(self, texts):
        """Extract features for all texts"""
        features_list = []
        for text in texts:
            try:
                features = self.extract_linguistic_features(str(text))
                features_list.append(features)
            except Exception as e:
                print(f"Error processing text: {e}")
                # Create default features in case of error
                features = {
                    'char_count': 0, 'word_count': 0, 'sentence_count': 1,
                    'avg_word_length': 0, 'avg_sentence_length': 0,
                    'flesch_reading_ease': 0, 'flesch_kincaid_grade': 0,
                    'exclamation_ratio': 0, 'question_ratio': 0, 'comma_ratio': 0,
                    'bullet_points': 0, 'sentiment_pos': 0, 'sentiment_neg': 0,
                    'sentiment_neu': 0, 'sentiment_compound': 0,
                    'first_person_pronouns': 0, 'passive_voice_indicators': 0,
                    'academic_phrases': 0
                }
                features_list.append(features)

        return pd.DataFrame(features_list)

# **Step 3: Text Vectorization**


In [ ]:
def create_text_features(texts, max_features=5000, ngram_range=(1, 2), vectorizer_type='tfidf'):
    """
    Create text features using different vectorization methods
    """
    # Clean and preprocess texts
    cleaned_texts = []
    for text in texts:
        if text and isinstance(text, str) and not pd.isna(text):
            cleaned_texts.append(str(text).strip())
        else:
            cleaned_texts.append("")

    # Choose vectorizer
    if vectorizer_type == 'tfidf':
        vectorizer = TfidfVectorizer(
            max_features=max_features,
            ngram_range=ngram_range,
            stop_words='english',
            lowercase=True,
            strip_accents='ascii',
            min_df=2,
            max_df=0.95
        )
    elif vectorizer_type == 'count':
        vectorizer = CountVectorizer(
            max_features=max_features,
            ngram_range=ngram_range,
            stop_words='english',
            lowercase=True,
            strip_accents='ascii',
            min_df=2,
            max_df=0.95
        )

    text_features = vectorizer.fit_transform(cleaned_texts)
    return text_features, vectorizer

# **Step 4: Model Training Pipeline**

In [ ]:
class FlexibleAIHumanClassifier:
    """
    Flexible classifier that can use any sklearn classifier with cross-validation training
    """

    def __init__(self, classifier_name='logistic', max_features=3000, ngram_range=(1, 2),
                 vectorizer_type='tfidf', use_scaling=True, cv_folds=5):
        self.feature_extractor = ReviewFeatureExtractor()
        self.vectorizer = None
        self.scaler = StandardScaler() if use_scaling else None
        self.use_scaling = use_scaling
        self.max_features = max_features
        self.ngram_range = ngram_range
        self.vectorizer_type = vectorizer_type
        self.classifier_name = classifier_name
        self.cv_folds = cv_folds
        self.cv_results = None

        # Initialize the classifier based on name
        self.model = self._get_classifier(classifier_name)

    def _get_classifier(self, classifier_name):
        """Get classifier by name with optimized parameters"""
        classifiers = {
            'logistic': LogisticRegression(random_state=42, max_iter=1000),
            'random_forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
            'svm_linear': LinearSVC(C=1.0,max_iter=1000,tol=1e-3,dual=False,random_state=4),
            'naive_bayes_multinomial': MultinomialNB(),
            'knn': KNeighborsClassifier(n_neighbors=5),
            'decision_tree': DecisionTreeClassifier(random_state=42),
            'extra_trees': ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=-1),
            'lda': LinearDiscriminantAnalysis(),
            'xgboost': XGBClassifier(random_state=42, n_jobs=-1),
            'mlp': MLPClassifier(random_state=42)
        }

        if classifier_name not in classifiers:
            raise ValueError(f"Classifier '{classifier_name}' not supported. Available: {list(classifiers.keys())}")

        return classifiers[classifier_name]

    def prepare_features(self, texts, fit_transform=True):
        """Prepare combined features (linguistic + text)"""
        # Extract linguistic features
        linguistic_features = self.feature_extractor.extract_all_features(texts)

        # Extract text features
        if fit_transform:
            text_features, self.vectorizer = create_text_features(
                texts, self.max_features, self.ngram_range, self.vectorizer_type
            )
        else:
            text_features = self.vectorizer.transform(texts)

        # Scale linguistic features if needed, but not for MultinomialNB
        if self.use_scaling and self.scaler and self.classifier_name != 'naive_bayes_multinomial':
            linguistic_features_scaled = self.scaler.fit_transform(linguistic_features.values) if fit_transform else self.scaler.transform(linguistic_features.values)
        else:
            linguistic_features_scaled = linguistic_features.values

        # Handle different classifiers that need dense vs sparse matrices
        if self.classifier_name == 'naive_bayes_multinomial':
            combined_features = text_features
        elif self.classifier_name == 'svm_linear' and not self.use_scaling:
            # Keep everything sparse
            linguistic_sparse = sparse.csr_matrix(linguistic_features_scaled)
            combined_features = sparse.hstack([linguistic_sparse, text_features])
        else:
            # For classifiers that require dense input
            text_dense = text_features.toarray()
            combined_features = np.hstack([linguistic_features_scaled, text_dense])

        return combined_features

    def train(self, X_train, y_train):
        """Train the classifier with cross-validation"""
        # Prepare features
        X_train_features = self.prepare_features(X_train, fit_transform=True)

        # Define scoring metrics
        scoring = ['accuracy', 'precision', 'recall', 'f1']

        # Perform cross-validation to evaluate performance
        self.cv_results = cross_validate(
            self.model, X_train_features, y_train,
            cv=self.cv_folds, scoring=scoring,
            return_train_score=True
        )

        # Fit the model on the entire training set
        self.model.fit(X_train_features, y_train)

        return self

    def get_cv_results(self):
        """Get cross-validation results"""
        if self.cv_results is None:
            return None

        results_summary = {}
        for metric in ['test_accuracy', 'test_precision', 'test_recall', 'test_f1']:
            scores = self.cv_results[metric]
            results_summary[metric] = {
                'mean': scores.mean(),
                'std': scores.std(),
                'scores': scores
            }
        return results_summary

    def predict(self, X_test):
        """Make predictions"""
        # Prepare features
        X_test_features = self.prepare_features(X_test, fit_transform=False)

        # Predict
        predictions = self.model.predict(X_test_features)

        # Get probabilities if available
        try:
            probabilities = self.model.predict_proba(X_test_features)
        except:
            probabilities = None

        return predictions, probabilities

    def evaluate(self, X_test, y_test):
        """Evaluate the model"""
        predictions, probabilities = self.predict(X_test)

        accuracy = accuracy_score(y_test, predictions)
        report = classification_report(y_test, predictions, target_names=['AI', 'Human'])
        conf_matrix = confusion_matrix(y_test, predictions)
        precision = precision_score(y_test, predictions)
        recall = recall_score(y_test, predictions)
        f1 = f1_score(y_test, predictions)

        # Calculate AUC if probabilities are available
        auc_score = None
        if probabilities is not None:
            auc_score = roc_auc_score(y_test, probabilities[:, 1])
            # Plot ROC curve
            fpr, tpr, thresholds = roc_curve(y_test, probabilities[:, 1])
            plt.figure()
            plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {auc_score:.2f})')
            plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver Operating Characteristic (ROC)')
            plt.legend(loc="lower right")
            plt.show()

        return {
            'accuracy': accuracy,
            'auc_score': auc_score,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'classification_report': report,
            'confusion_matrix': conf_matrix,
            'predictions': predictions,
            'probabilities': probabilities
        }

# **Step 6: Usage Example**

In [ ]:
df = load_and_prepare_data('/content/subtaskC_train_dev.jsonl')
X = df['text'].values
y = df['label'].values

38906


**Logistic Regression**

In [ ]:
classifier = FlexibleAIHumanClassifier(classifier_name='logistic', cv_folds=5)
classifier.train(X, y)

In [ ]:
cv_results = classifier.get_cv_results()
for metric, values in cv_results.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9588 +/- 0.0055
test_precision: 0.9602 +/- 0.0072
test_recall: 0.9573 +/- 0.0066
test_f1: 0.9587 +/- 0.0055


In [ ]:
logistic10 = FlexibleAIHumanClassifier(classifier_name='logistic', cv_folds=10)
logistic10.train(X, y)

In [ ]:
cv_results10 = logistic10.get_cv_results()
for metric, values in cv_results10.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9685 +/- 0.0326
test_precision: 0.9556 +/- 0.0532
test_recall: 0.9871 +/- 0.0110
test_f1: 0.9702 +/- 0.0284


**MLP**

In [ ]:
mlp = FlexibleAIHumanClassifier(classifier_name='mlp', cv_folds=5)
mlp.train(X, y)


In [ ]:
mlp_cv_results = mlp.get_cv_results()
for metric, values in mlp_cv_results.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9788 +/- 0.0131
test_precision: 0.9728 +/- 0.0169
test_recall: 0.9858 +/- 0.0283
test_f1: 0.9789 +/- 0.0133


In [ ]:
mlp10 = FlexibleAIHumanClassifier(classifier_name='mlp', cv_folds=10)
mlp10.train(X, y)


In [ ]:
mlp_cv_results10 = mlp10.get_cv_results()
for metric, values in mlp_cv_results10.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9874 +/- 0.0138
test_precision: 0.9784 +/- 0.0248
test_recall: 0.9977 +/- 0.0067
test_f1: 0.9877 +/- 0.0130


**Random forest**

In [ ]:
random_forest = FlexibleAIHumanClassifier(classifier_name='random_forest', cv_folds=5)
random_forest.train(X, y)

In [ ]:
random_forest_cv_results = random_forest.get_cv_results()
for metric, values in random_forest_cv_results.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9714 +/- 0.0442
test_precision: 0.9889 +/- 0.0139
test_recall: 0.9543 +/- 0.0910
test_f1: 0.9687 +/- 0.0498


In [ ]:
random_forest10 = FlexibleAIHumanClassifier(classifier_name='random_forest', cv_folds=10)
random_forest10.train(X, y)

In [ ]:
random_forest_cv_results10 = random_forest10.get_cv_results()
for metric, values in random_forest_cv_results10.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9926 +/- 0.0113
test_precision: 0.9911 +/- 0.0175
test_recall: 0.9946 +/- 0.0153
test_f1: 0.9927 +/- 0.0112


**Naive Bayes**

In [ ]:
naive_bayes = FlexibleAIHumanClassifier(classifier_name='naive_bayes_multinomial', cv_folds=5)
naive_bayes.train(X, y)

In [ ]:
naive_bayes_cv_results = naive_bayes.get_cv_results()
for metric, values in naive_bayes_cv_results.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9115 +/- 0.0352
test_precision: 0.8632 +/- 0.0507
test_recall: 0.9839 +/- 0.0044
test_f1: 0.9187 +/- 0.0288


In [ ]:
naive_bayes10 = FlexibleAIHumanClassifier(classifier_name='naive_bayes_multinomial', cv_folds=10)
naive_bayes10.train(X, y)

In [ ]:
naive_bayes_cv_results10 = naive_bayes10.get_cv_results()
for metric, values in naive_bayes_cv_results10.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9167 +/- 0.0481
test_precision: 0.8724 +/- 0.0645
test_recall: 0.9856 +/- 0.0033
test_f1: 0.9241 +/- 0.0380


KNN

In [ ]:
knn = FlexibleAIHumanClassifier(classifier_name='knn', cv_folds=5)
knn.train(X, y)


4304


In [ ]:
knn_cv_results = knn.get_cv_results()
for metric, values in knn_cv_results.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9008 +/- 0.0436
test_precision: 0.8639 +/- 0.0326
test_recall: 0.9543 +/- 0.0888
test_f1: 0.9044 +/- 0.0474


In [ ]:
knn10= FlexibleAIHumanClassifier(classifier_name='knn', cv_folds=10)
knn10.train(X, y)


In [ ]:
knn_cv_results10 = knn10.get_cv_results()
for metric, values in knn_cv_results10.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9186 +/- 0.0458
test_precision: 0.8803 +/- 0.0407
test_recall: 0.9723 +/- 0.0823
test_f1: 0.9217 +/- 0.0489


**Decision Tree**

In [ ]:
desicion_tree = FlexibleAIHumanClassifier(classifier_name='decision_tree', cv_folds=5)
desicion_tree.train(X, y)

In [ ]:
desicion_tree_results = desicion_tree.get_cv_results()
for metric, values in desicion_tree_results.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9443 +/- 0.0447
test_precision: 0.9378 +/- 0.0261
test_recall: 0.9535 +/- 0.0926
test_f1: 0.9428 +/- 0.0501


In [ ]:
desicion_tree10 = FlexibleAIHumanClassifier(classifier_name='decision_tree', cv_folds=10)
desicion_tree10.train(X, y)

In [ ]:
desicion_tree_results10 = desicion_tree10.get_cv_results()
for metric, values in desicion_tree_results10.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9695 +/- 0.0185
test_precision: 0.9480 +/- 0.0306
test_recall: 0.9951 +/- 0.0141
test_f1: 0.9706 +/- 0.0169


**Extra Trees**

In [ ]:
extra_trees = FlexibleAIHumanClassifier(classifier_name='extra_trees', cv_folds=5)
extra_trees.train(X, y)

In [ ]:
extra_trees_results = extra_trees.get_cv_results()
for metric, values in extra_trees_results.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9706 +/- 0.0518
test_precision: 0.9942 +/- 0.0077
test_recall: 0.9471 +/- 0.1053
test_f1: 0.9667 +/- 0.0596


In [ ]:
extra_trees10 = FlexibleAIHumanClassifier(classifier_name='extra_trees', cv_folds=10)
extra_trees10.train(X, y)

In [ ]:
extra_trees_results10 = extra_trees10.get_cv_results()
for metric, values in extra_trees_results10.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9940 +/- 0.0109
test_precision: 0.9954 +/- 0.0090
test_recall: 0.9928 +/- 0.0208
test_f1: 0.9940 +/- 0.0111


**LDA**

In [ ]:
lda = FlexibleAIHumanClassifier(classifier_name='lda', cv_folds=5)
lda.train(X, y)

In [ ]:
lda_results = lda.get_cv_results()
for metric, values in lda_results.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9587 +/- 0.0201
test_precision: 0.9432 +/- 0.0219
test_recall: 0.9774 +/- 0.0392
test_f1: 0.9594 +/- 0.0207


In [ ]:
lda10 = FlexibleAIHumanClassifier(classifier_name='lda', cv_folds=10)
lda10.train(X, y)

In [ ]:
lda_results10 = lda10.get_cv_results()
for metric, values in lda_results10.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

**xgboost**

In [ ]:
xgboost = FlexibleAIHumanClassifier(classifier_name='xgboost', cv_folds=5)
xgboost.train(X, y)

In [ ]:
xgboost_results = xgboost.get_cv_results()
for metric, values in xgboost_results.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9745 +/- 0.0170
test_precision: 0.9680 +/- 0.0264
test_recall: 0.9827 +/- 0.0296
test_f1: 0.9748 +/- 0.0169


In [ ]:
xgboost10 = FlexibleAIHumanClassifier(classifier_name='xgboost', cv_folds=10)
xgboost10.train(X, y)

In [ ]:
xgboost_results10 = xgboost10.get_cv_results()
for metric, values in xgboost_results10.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

***SVM***

In [ ]:
svm = FlexibleAIHumanClassifier(classifier_name='svm_linear', cv_folds=5)
svm.train(X, y)

In [ ]:
svm_results = svm.get_cv_results()
for metric, values in svm_results.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")

test_accuracy: 0.9736 +/- 0.0139
test_precision: 0.9637 +/- 0.0212
test_recall: 0.9851 +/- 0.0286
test_f1: 0.9739 +/- 0.0140


In [ ]:
svm10 = FlexibleAIHumanClassifier(classifier_name='svm_linear', cv_folds=10)
svm10.train(X, y)

In [ ]:
svm_results10 = svm10.get_cv_results()
for metric, values in svm_results10.items():
    print(f"{metric}: {values['mean']:.4f} +/- {values['std']:.4f}")